In [1]:
import matplotlib.pyplot as plt
import sqlalchemy as sq
import seaborn as sns
import pandas as pd
import numpy as np
import os, sys

sys.path.append("../")
from Shared.DataService import DataService

In [ ]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [ ]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")

db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
conn = db.connect()

In [2]:
dlyDF = pd.read_csv('data/aggregatedDly.csv')

In [ ]:
ergotQuery = sq.text(
    """
    SELECT percnt_true, has_ergot, sum_severity, present_prev1, present_prev2, present_prev3, present_in_neighbor, severity_prev1, 
        severity_prev2, severity_prev3, severity_in_neighbor FROM public.agg_ergot_sample
    GROUP BY year, district, percnt_true, has_ergot, sum_severity, present_prev1, present_prev2, present_prev3, present_in_neighbor, severity_prev1, 
        severity_prev2, severity_prev3, severity_in_neighbor;
    """
)

ergotDF = pd.read_sql_query(ergotQuery, conn)

In [ ]:
final_df = dlyDF.merge(ergotDF, on=["year", "district"])

In [3]:
# Basic correlogram
sns.pairplot(df)
plt.show()

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x0000025C5B648AF0> (for post_execute):


ValueError: Image size of 1369500x1369500 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 1369500x1369500 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1.3695e+06x1.3695e+06 with 14335 Axes>

In [ ]:
sns.set_theme(style="white")

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})